## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi</a> 2024/7/8

### 这是<a href="https://www.kaggle.com/competitions/m5-forecasting-accuracy">M5 forecast</a>比赛,这里学习的是<a href="https://www.kaggle.com/code/mayer79/m5-forecast-poisson-loss-top-10">M5 Forecast: Poisson Loss (top 10%)</a>的方案,经过我的改进在排行榜上能达到395名,参赛人数为5582名,即0.0707631673。
    

### 1.导入必要的python库。固定随机种子,保证模型可以复现。

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#矩阵运算与科学计算的库
#model lightgbm回归模型
import lightgbm as lgb
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

### 2.导入数据集。

#### 这里有5个数据集,下面我们一个一个来看看。

### 2.1   calendar是日历文件。date是日期,wm_yr_wk是周的编号,因为每个相同的wm_yr_wk在数据中有7个。weekday和wday都是星期几,month是月,year是年,d是从训练数据开头开始的天数,总共有将近2000天。event_name和event_type可能是节假日,因此数据中大部分都是缺失值,snap_CA、snap_TX,snap_WI是其他的特征。

In [2]:
calendar=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
print(f"len(calendar):{len(calendar)}")
calendar.head()

len(calendar):1969


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


#### 这里先对日期数据进行处理,由于这里注释已经写的很详细了,这里不多解释。

In [3]:
from sklearn.preprocessing import OrdinalEncoder#对类别型变量进行编码的工具

#对日历这张表格的处理
def prep_calendar(df):
    #event_name和event_type缺失值个数一样,很有可能是同一个意义的变量
    #weekday和wday重复,wday已经是数值了,故drop weekday
    #date年和月有,天可能没啥用?
    df = df.drop(["date", "weekday", "event_type_1", "event_type_2"], axis=1)
    
    #'d'列从'd_1'变成1
    df = df.assign(d = df.d.str[2:].astype(int))
    
    #缺失值>0.9的两列,但仍有信息
    to_ordinal = ["event_name_1", "event_name_2"]
    #由于缺失值占大多数,所以先将缺失值填充为1,然后剩下的按出现的先后顺序标记为2,3,4,5……
    df[to_ordinal] = df[to_ordinal].fillna("1")
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1
    
    #可能是为了节省内存,这些本来就是数值列
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")
    
    return df

calendar = prep_calendar(calendar)
calendar.head()

,wm_yr_wk,wday,month,year,d,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI
0,11101,1,1,2011,1,1,1,0,0,0
1,11101,2,1,2011,2,1,1,0,0,0
2,11101,3,1,2011,3,1,1,0,0,0
3,11101,4,2,2011,4,1,1,1,1,0
4,11101,5,2,2011,5,1,1,1,0,1


### 2.2 validation和evaluation

#### 这两个文件要一起看,validation是day1到day1913预测接下来28天,这也是公榜的成绩;evaluation是day1到day1941预测接下来28天,这也是私榜的成绩。由于evaluation已经给出了公榜的target,所以公榜前排都是0。除了每天的数据以外,剩下都是一些id。

In [4]:
train_validation=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
print(f"len(train_validation):{len(train_validation)}")
train_validation.head()

len(train_validation):30490


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
train_evaluation=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
print(f"len(train_evaluation):{len(train_evaluation)}")
train_evaluation.head()

len(train_evaluation):30490


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


### 2.3 这是价格的表,就是在第wm_yr_wk周在store_id买item_id的价格为sell_price。

In [6]:
sales=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
print(f"len(sales):{len(sales)}")
sales.head()

len(sales):6841121


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


### 2.4提交文件,看看提交的格式。

In [7]:
submission=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
print(f"len(submission):{len(submission)}")
submission.head()

len(submission):60980


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.特征工程。

#### 特征工程就是下面的这段代码,看起来代码量不多,特征也是熟悉的滞后特征和滑动窗口。不知道为什么效果这么好。

In [8]:
LAGS = [7, 28]#7是一周,28是4周约等于一个月
WINDOWS = [7, 28]#7是一周,28是4周约等于一个月
FIRST = 1942#测试集的第一天
LENGTH = 28#测试集的天数

#对于需求特征的构造
def demand_features(df):
    for lag in LAGS:
        #前lag天的demand
        df[f'lag_t{lag}'] = df.groupby('id')['demand'].transform(lambda x: x.shift(lag)).astype("float32")
        for w in WINDOWS:
            #前lag天到前lag+w天的均值
            df[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id')[f'lag_t{lag}'].transform(lambda x: x.rolling(w).mean()).astype("float32")
    return df

#### 这里是将数据划分为训练集,验证集和测试集,这里的代码和特征工程关系不大,更多的是对数据进行一些处理,例如rename,drop。这里的数据是时序数据,但是他这里随机划分,所以肯定存在着数据泄露,这也是为什么他在验证集上的分数为0.

In [9]:
from sklearn.model_selection import train_test_split
def prep_data(df, drop_d):
    
    #去掉前1000-28天,可能是时序数据距离测试集太远,相关性不高了
    df = df.drop(["d_" + str(i+1) for i in range(drop_d)], axis=1)

    # id例如'HOBBIES_1_005_CA_1_evaluation'变成‘HOBBIES_1_005_CA_1’
    df = df.assign(id=df.id.str.replace("_evaluation", ""))
    
    #这是增加了要预测的几列,当然这里全部都是缺失值
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(FIRST + i) for i in range(LENGTH)])
    
    #熔化:比如有['id','d1','d2','d3']的一行数据,就会变成['id','d','demand']的3行数据
    #demand的数值分别为d1,d2,d3,d列就是['d1','d2','d3']
    df = df.melt(id_vars=["id", "item_id", "store_id", "state_id", "dept_id", "cat_id"], var_name='d', value_name='demand')
    
    #这里将'd_999'变成数值,并使用int16,因为day的范围都在2000以内。浮点数用float32也足够精确了
    df = df.assign(d=df.d.str[2:].astype("int16"),
                   demand=df.demand.astype("float32"))
    
    #对于需求特征的构造,这里很常规
    df = demand_features(df)
    
    #有些shift特征构造的时候会有缺失值,去掉有缺失值的列
    df = df[df.d > (drop_d + max(LAGS) + max(WINDOWS))]
    
    #将日历特征按照天进行拼接,并且得到了时间特征wm_yr_wk	wday,month,year,d
    df = df.merge(calendar, how="left", on="d")
    #根据store_id,item_id和wm_yr_wk将sales特征加入
    df = df.merge(sales, how="left", on=["store_id", "item_id", "wm_yr_wk"])
    #这个编号貌似是周,因为每个相同编号的个数都为7,去掉这个是因为有wday。
    df = df.drop(["wm_yr_wk"], axis=1)
    
    #类别型变量按照出现的先后顺序变成[0,1,2,……]
    for v in ["item_id", "store_id", "state_id", "dept_id", "cat_id"]:
        df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]]).astype("int16") + 1
    
    #id就是一个字符串编号,d是单调递增的天数,demand是要预测target,这些都不能作为Input.
    x = list(set(df.columns) - {'id', 'd', 'demand'})
            
    #测试集选的有点大,相当于3倍的测试数据+真正没有标签的测试数据
    test = df[df.d >= FIRST - max(LAGS) - max(WINDOWS) - LENGTH]
    #这里是把官方测试数据移除了
    df = df[df.d < FIRST]
    #用训练数据随机划分x和y,由于是随机划分,肯定也会存在数据穿越的问题。
    xtrain, xvalid, ytrain, yvalid = train_test_split(df[x], df["demand"], test_size=0.1, shuffle=True, random_state=2024)
    #按照9:1的比例构造了训练数据和验证数据
    train = lgb.Dataset(xtrain, label = ytrain)
    valid = lgb.Dataset(xvalid, label = yvalid)

    return train, valid, test, x
#对于训练数据按照9:1的比例划分成训练集和验证集，测试集选的有点大,相当于3倍的测试数据+真正没有标签的测试数据,x是特征
train, valid, test, x = prep_data(train_evaluation, 1000 - LENGTH)
test.head()

,id,item_id,store_id,state_id,dept_id,cat_id,d,demand,lag_t7,rolling_mean_lag7_w7,...,rolling_mean_lag28_w28,wday,month,year,event_name_1,event_name_2,snap_CA,snap_TX,snap_WI,sell_price
25276210,HOBBIES_1_001_CA_1,1438,1,1,4,2,1858,0.0,4.0,1.142857,...,0.785714,3,2,2016,1,1,0,0,0,8.26
25276211,HOBBIES_1_002_CA_1,1439,1,1,4,2,1858,0.0,0.0,0.285714,...,0.178571,3,2,2016,1,1,0,0,0,3.97
25276212,HOBBIES_1_003_CA_1,1440,1,1,4,2,1858,0.0,0.0,0.428571,...,0.107143,3,2,2016,1,1,0,0,0,2.97
25276213,HOBBIES_1_004_CA_1,1441,1,1,4,2,1858,0.0,2.0,1.857143,...,2.071429,3,2,2016,1,1,0,0,0,4.64
25276214,HOBBIES_1_005_CA_1,1442,1,1,4,2,1858,1.0,0.0,0.857143,...,0.750000,3,2,2016,1,1,0,0,0,2.88


### 4.模型的训练

#### 这里我没有见过的参数就是'objective': 'poisson',这是使用泊松回归,适用于计数数据的回归问题。

In [10]:
#用训练集和验证集训练模型
def fit_model(train, valid):
    params = {
        'metric': 'rmse',
        'objective': 'poisson',
        'seed': 2024,#随机数种子
        'force_row_wise' : True,#按行读取数据
        'learning_rate' : 0.08,#学习率
        'lambda': 0.1,
        'num_leaves': 63,
        'sub_row' : 0.7,
        'bagging_freq' : 1,
        'colsample_bytree': 0.7
    }

    fit = lgb.train(params, train, 
                    num_boost_round =2000, 
                    valid_sets = [valid]
                   )
    return fit
#这里根据训练集和验证集训练了模型
fit = fit_model(train, valid)

[LightGBM] [Info] Total Bins 1781
[LightGBM] [Info] Number of data points in the train set: 25053633, number of used features: 20
[LightGBM] [Info] Start training from score 0.205892


### 5.模型的推理和预测结果的保存。

#### 我的重点放在特征工程的学习上,这里就没怎么看了。感兴趣的话可以研究一下。

In [11]:
def demand_features_eval(df):
    #每个id都取最后一天的数据(这里其实就是测试数据的FIRST到FIRST+LENGTH)
    return df.groupby('id', sort=False).last().reset_index()

def pred_all(fit, test, x):
    
    for day in range(FIRST, FIRST + LENGTH):
        test_day = demand_features_eval(test[(test.d <= day) & (test.d >= day - max(LAGS) - max(WINDOWS))])
        test.loc[test.d == day, "demand"] = fit.predict(test_day[x])
    return test

def pred_to_csv(test, cols, file):
     
    #Prepare for reshaping
    test = test.assign(id=test.id + "_" + np.where(test.d < FIRST, "validation", "evaluation"),
                       F="F" + (test.d - FIRST + LENGTH + 1 - LENGTH * (test.d >= FIRST)).astype("str"))
    
    #Reshape
    submission = test.pivot(index="id", columns="F", values="demand").reset_index()[cols].fillna(1)
    
    #Export
    submission.to_csv(file, index=False)
    
pred = pred_all(fit, test, x)
pred_to_csv(pred, cols=submission.columns, file="submission.csv")